# Tutorial de uso para crear scripts y notebooks de python (VQE/VQD)

La idea de este tutorial es indicar el flujo de trabajo para poder usar las funcionalidad definidas. Todo lo anterior puede ser usado para scripts o para trabajar en jupyters.

Hecho por: Javier Norambuena Leiva

In [1]:
import quantumsim as qs

## Definir sistema

El primer paso es definir el sistema a trabajar ....

## Definir ansatz

## Definir optimizador

# Tutorial de uso para crear scripts y notebooks para ansatz adaptativo